<a href="https://colab.research.google.com/github/whiteevl/machine-learning/blob/main/Proyek_Akhir_Klasifikasi_Gambar_Ardey%20dicoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

!apt-get install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=30dc47b28fbec6381a6111a751026bbc283858c114318f6a2bfd2848ca96b56b
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [2]:
import wget
import zipfile
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2


In [3]:
url = 'https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip'

wget.download(url, 'rockpaperscissors.zip')


'rockpaperscissors.zip'

In [4]:
with zipfile.ZipFile('rockpaperscissors.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [5]:
data_dir = 'rockpaperscissors'
train_dir = 'train'
val_dir = 'val'

In [6]:
classes = ['rock', 'paper', 'scissors']


In [7]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [8]:
for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    images = os.listdir(cls_dir)
    train_images, val_images = train_test_split(images, test_size=0.4, random_state=42)

    train_cls_dir = os.path.join(train_dir, cls)
    val_cls_dir = os.path.join(val_dir, cls)

    os.makedirs(train_cls_dir, exist_ok=True)
    os.makedirs(val_cls_dir, exist_ok=True)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dest = os.path.join(train_cls_dir, img)
        shutil.copy(src, dest)

    for img in val_images:
        src = os.path.join(cls_dir, img)
        dest = os.path.join(val_cls_dir, img)
        shutil.copy(src, dest)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [10]:
batch_size = 32
img_size = (150, 150)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [12]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

9406464/9406464 [==============================] - 0s 0us/step


In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [14]:
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, verbose=1)


In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[checkpoint]
)

Epoch 1/15
41/41 [==============================] - ETA: 0s - loss: 0.4000 - accuracy: 0.8674
Epoch 1: val_loss improved from inf to 0.06113, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


41/41 [==============================] - 52s 1s/step - loss: 0.4000 - accuracy: 0.8674 - val_loss: 0.0611 - val_accuracy: 0.9817
Epoch 2/15
41/41 [==============================] - ETA: 0s - loss: 0.0782 - accuracy: 0.9710
Epoch 2: val_loss improved from 0.06113 to 0.04740, saving model to best_model.h5
41/41 [==============================] - 46s 1s/step - loss: 0.0782 - accuracy: 0.9710 - val_loss: 0.0474 - val_accuracy: 0.9863
Epoch 3/15
41/41 [==============================] - ETA: 0s - loss: 0.0717 - accuracy: 0.9756
Epoch 3: val_loss did not improve from 0.04740
41/41 [==============================] - 39s 965ms/step - loss: 0.0717 - accuracy: 0.9756 - val_loss: 0.0524 - val_accuracy: 0.9829
Epoch 4/15
41/41 [==============================] - ETA: 0s - loss: 0.0814 - accuracy: 0.9710
Epoch 4: val_loss did not improve from 0.04740
41/41 [==============================] - 38s 941ms/step - loss: 0.0814 - accuracy: 0.9710 - val_loss: 0.0534 - val_accuracy: 0.9829
Epoch 5/15
41/41 [==

In [16]:
val_loss, val_acc = model.evaluate(val_generator, verbose=2)
print(f'Val accuracy: {val_acc}')

28/28 - 13s - loss: 0.0670 - accuracy: 0.9840 - 13s/epoch - 478ms/step
Val accuracy: 0.9840182662010193


In [17]:
nama = "Aradea Bayu Murti"
email = "whiteevilfero@gmail.com"
no_hp = "6281649488239"
id_dicoding = "priceofdarkness"

print(f'Nama: {nama}')
print(f'Email: {email}')
print(f'No. HP: {no_hp}')
print(f'ID Dicoding: {id_dicoding}')

Nama: Aradea Bayu Murti
Email: whiteevilfero@gmail.com
No. HP: 6281649488239
ID Dicoding: priceofdarkness


In [19]:
from google.colab import files
uploaded = files.upload()

Saving WhatsApp Image 2023-03-10 at 9.59.57 PM.jpeg to WhatsApp Image 2023-03-10 at 9.59.57 PM.jpeg


In [20]:
import os
os.listdir()

['.config',
 'best_model.h5',
 'val',
 'rockpaperscissors',
 'WhatsApp Image 2023-03-10 at 9.59.57 PM.jpeg',
 'rockpaperscissors.zip',
 'train',
 '__MACOSX',
 'sample_data']

In [21]:
image_path = 'WhatsApp Image 2023-03-10 at 9.59.57 PM.jpeg'
img = image.load_img(image_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale pixel values to [0, 1]
prediction = model.predict(img_array)
predicted_class = classes[np.argmax(prediction)]

print(f'Hasil prediksi: {predicted_class}')

1/1 [==============================] - 1s 1s/step
Hasil prediksi: paper
